# 目的
- 全てのデータを該当タグごとにテーブルにする
- 取れていない or 過剰に取れたデータが存在する場合に、それがOriginalにあるか、外部データにあるか、を一眼で判断できるようにする

In [1]:
import polars as pl
import matplotlib.pyplot as plt
import ast

import itertools
from collections import Counter

In [4]:
pii_dataset_fixed

document,text,tokens,trailing_whitespace,labels
str,str,list[str],list[bool],list[str]
"""001010b6-de5c-…","""In my role as …","[""In"", ""my"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""0016aef8-0d26-…","""Hello, my name…","[""Hello"", "","", … "".""]","[false, true, … false]","[""O"", ""O"", … ""O""]"
"""00207a69-33cc-…","""My name is Sof…","[""My"", ""name"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""0036e5b9-031d-…","""In a bustling …","[""In"", ""a"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""004467fb-d131-…","""In 2018, as a …","[""In"", ""2018"", … "")""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""004a6d38-9a0f-…","""In 2012, I was…","[""In"", ""2012"", … ""Surfing""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""005acbda-4d4f-…","""As Victoria Wi…","[""As"", ""Victoria"", … "".""]","[true, true, … false]","[""O"", ""B-NAME_STUDENT"", … ""O""]"
"""00622bda-4d00-…","""My name is Raj…","[""My"", ""name"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""00661376-ca29-…","""During my tenu…","[""During"", ""my"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"


In [2]:
train = pl.read_json("../data/train.json")

pii_dataset_fixed = (
    pl.scan_csv("../data/pii_dataset_fixed.csv")
    .with_columns(
        pl.col("tokens").map_elements(lambda x: ast.literal_eval(x)),
        pl.col("trailing_whitespace").map_elements(lambda x: ast.literal_eval(x)),
        pl.col("labels").map_elements(lambda x: ast.literal_eval(x)),
    )
    .collect()
)

moredata_dataset_fixed = (
    pl.scan_csv("../data/moredata_dataset_fixed.csv")
    .with_columns(
        pl.col("tokens").map_elements(lambda x: ast.literal_eval(x)),
        pl.col("trailing_whitespace").map_elements(lambda x: ast.literal_eval(x)),
        pl.col("labels").map_elements(lambda x: ast.literal_eval(x)),
    )
    .collect()
)

In [3]:
moredata_dataset_fixed

document,text,tokens,trailing_whitespace,labels
str,str,list[str],list[bool],list[str]
"""pj_0""","""In today's mod…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""pj_1""","""In today's mod…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""pj_10""","""The Importance…","[""The"", ""Importance"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""pj_100""","""The Importance…","[""The"", ""Importance"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""pj_1000""","""The Importance…","[""The"", ""Importance"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""pj_1001""","""In today's fas…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""pj_1002""","""In today's mod…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""pj_1003""","""Monica: A Stud…","[""Monica"", "":"", … "".""]","[false, true, … false]","[""B-NAME_STUDENT"", ""O"", … ""O""]"
"""pj_1004""","""In today's mod…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"


In [ ]:
all_entities = [
    "B-NAME_STUDENT",
    "I-NAME_STUDENT",
    "B-URL_PERSONAL",
    "I-URL_PERSONAL",
    "B-EMAIL",
    "B-ID_NUM",
    "I-ID_NUM",
    "B-USERNAME",
    "B-PHONE_NUM",
    "I-PHONE_NUM",
    "B-STREET_ADDRE",
    "I-STREET_ADDRE",
]

In [11]:
for tokens, labels in zip(train["tokens"], train["labels"]):
    for token, label in zip(tokens, labels):
        if label == "B-NAME_STUDENT":
            b_name_students.append()

document,full_text,tokens,trailing_whitespace,labels
i64,str,list[str],list[bool],list[str]
7,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
10,"""Diego Estrada …","[""Diego"", ""Estrada"", … "" ""]","[true, false, … false]","[""B-NAME_STUDENT"", ""I-NAME_STUDENT"", … ""O""]"
16,"""Reporting proc…","[""Reporting"", ""process"", … "" ""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
20,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
56,"""Assignment: V…","[""Assignment"", "":"", … "" ""]","[false, false, … false]","[""O"", ""O"", … ""O""]"
86,"""Cheese Startup…","[""Cheese"", ""Startup"", … "" ""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
93,"""Silvia Villalo…","[""Silvia"", ""Villalobos"", … "" ""]","[true, false, … false]","[""B-NAME_STUDENT"", ""I-NAME_STUDENT"", … ""O""]"
104,"""Storytelling …","[""Storytelling"", "" "", … "" ""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
112,"""Reflection – L…","[""Reflection"", ""–"", … "" ""]","[true, true, … false]","[""O"", ""O"", … ""O""]"


document,text,tokens,trailing_whitespace,labels
str,str,list[str],list[bool],list[str]
"""pj_0""","""In today's mod…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""pj_1""","""In today's mod…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""pj_10""","""The Importance…","[""The"", ""Importance"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""pj_100""","""The Importance…","[""The"", ""Importance"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""pj_1000""","""The Importance…","[""The"", ""Importance"", … "".""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
"""pj_1001""","""In today's fas…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""pj_1002""","""In today's mod…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
"""pj_1003""","""Monica: A Stud…","[""Monica"", "":"", … "".""]","[false, true, … false]","[""B-NAME_STUDENT"", ""O"", … ""O""]"
"""pj_1004""","""In today's mod…","[""In"", ""today"", … "".""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
